In [1]:
# 코드 실행 후 마지막 ngrok url을 4_multi-llm-gradio.ipynb에 적용

In [1]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:2 https://download.docker.com/linux/ubuntu focal InRelease                 
Hit:3 https://cli.github.com/packages stable InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease3m
Hit:9 https://ngrok-agent.s3.amazonaws.com buster InRelease                    
Reading package lists... Done
Building dependency tree       
Reading state information... Done
13 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependenc

In [2]:
!pip install -q vllm fastapi uvicorn ngrok pyngrok nest_asyncio

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from vllm import LLM, SamplingParams
import uvicorn
import os
import torch
from transformers import AutoTokenizer
from pyngrok import ngrok
import nest_asyncio
from starlette.responses import StreamingResponse
import asyncio

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
ngrok.set_auth_token('2hgvs8djD2xuTeO4svNzxiIM7G4_7qyy1Jo94tfQ3Acvoba1t')

In [6]:
BASE_MODEL = "google/gemma-2b-it"
FINTUNED_MODEL = "bradmin/gemma-2b-persona-instruct"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

tensor_parallel_size = int(os.environ.get("DEVICES", "1"))

llm = LLM(
    model=FINTUNED_MODEL,
    dtype=torch.float16,
    # kv_cache_dtype="fp8",
    tensor_parallel_size=tensor_parallel_size,
    )

INFO 06-15 04:21:10 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='bradmin/gemma-2b-persona-instruct', speculative_config=None, tokenizer='bradmin/gemma-2b-persona-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=bradmin/gemma-2b-persona-instruct)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-15 04:21:11 selector.py:131] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-15 04:21:11 selector.py:51] Using XFormers backend.
INFO 06-15 04:21:12 selector.py:131] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-15 04:21:12 selector.py:51] Using XFormers backend.
WARNING 06-15 04:21:12 gemma.py:54] Gemma's activation function was incorrectly set to exact GeLU in the config JSON file when it was initially released. Changing the activation function to approximate GeLU (`gelu_pytorch_tanh`). If you want to use the legacy `gelu`, edit the config JSON to set `hidden_activation=gelu` instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
INFO 06-15 04:21:12 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 06-15 04:21:14 model_runner.py:160] Loading model weights took 4.6720 GB
INFO 06-15 04:21:16 gpu_executor.py:83] # GPU blocks: 23501, # CPU blocks: 14563
INFO 06-15 0

In [7]:
nest_asyncio.apply()

app = FastAPI()

formatted_prompt = """<bos><start_of_turn>user
당신은 유저의 대답을 듣고 그를 진정시킨 후, 그의 이야기가 얼마나 현실성 없는 이야기인지 대답해주는 모델입니다. 다양한 방식으로 그를 설득해 회사에 남아있고 그의 목표를 나중에 시도하라고 이야기 해주세요.

{}<end_of_turn>
<start_of_turn>model\n
"""

class CompletionRequest(BaseModel):
    prompt: str
    max_tokens: int
    min_tokens: int
    repetition_penalty: float
    temperature: float
    top_k: int
    top_p: float

@app.post("/v1/completions")
async def completions(request: CompletionRequest):
  sampling_params = SamplingParams(
      max_tokens=request.max_tokens,
      min_tokens=request.min_tokens,
      repetition_penalty=request.repetition_penalty,
      temperature=request.temperature,
      top_k=request.top_k,
      top_p=request.top_p,
      )
  prompt = formatted_prompt.format(request.prompt)
  print(prompt)
  response = llm.generate([prompt], sampling_params)
  ans = response[0].outputs[0].text
  return ans

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)

t=2024-06-15T04:21:28+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
INFO:     Started server process [10267]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://4f0f-44-211-63-248.ngrok-free.app" -> "http://localhost:8000"
